In [1]:
import os
import sys

MODULE_PATH = 'C:\Github\DL_Study\Base'
MODEL_PATH = 'C:\Github\DL_Study\RNN'

sys.path.insert(0, MODULE_PATH)
sys.path.insert(0, MODEL_PATH)

import numpy
from config import *
from seq2seq_np import *

In [2]:
# configuration setting
def model_config():
    # parameter for LSTM Model
    epochs = [30]
    batch_size = [64]
    learning_rate = [0.01, 0.001]
    
    # create config data
    configs = []
    for i in epochs:
        for j in batch_size:
            for k in learning_rate:
                config = [i, j, k]
                configs.append(config)
    return configs

# fucntion for fit cnn model using configs
def model_fit(train_X, train_y, config):
    # unpack config
    n_epochs, n_batch, learning_rate = config
    model = Seq2Seq(time_size=24, hidden_size=64, feature_size=17)
    # fit model and return
    model.fit(train_X=train_X, train_y=train_y, epochs=n_epochs, 
              batch_size=n_batch, learning_rate=learning_rate)
    return model

def MAE_metric(x, t):
    return np.mean(numpy.abs(x-t))

def MSE_metric(x, t):
    return np.mean((x-t)**2)

In [3]:
# dataset
import pandas as pd
import numpy
import time
from datetime import datetime

np.random.seed(42)
numpy.random.seed(42)

data_url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/00492/Metro_Interstate_Traffic_Volume.csv.gz'
df = pd.read_csv(data_url,compression='gzip', index_col=7)
df.drop(columns='weather_description', inplace=True)
df.head()

,holiday,temp,rain_1h,snow_1h,clouds_all,weather_main,traffic_volume
date_time,,,,,,,
2012-10-02 09:00:00,None,288.28,0.0,0.0,40,Clouds,5545
2012-10-02 10:00:00,None,289.36,0.0,0.0,75,Clouds,4516
2012-10-02 11:00:00,None,289.58,0.0,0.0,90,Clouds,4767
2012-10-02 12:00:00,None,290.13,0.0,0.0,90,Clouds,5026
2012-10-02 13:00:00,None,291.14,0.0,0.0,75,Clouds,4918


In [4]:
# series data to img function
def series_to_img(dataset, time_step=1):
    num = dataset.shape[1]      # features num
    df = pd.DataFrame(dataset)
    cols, names = list(), list()
    # sequence t-n to t-1
    for i in range(time_step, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j+1, i)) for j in range(num)]

    for i in range(0, 1):
        cols.append(df.shift(-i))
        if i == 0:
            names += [('var%d(t)' % (j+1)) for j in range(num)]
        else:
            names += [('var%d(t+%d)' % (j+1, i)) for j in range(num)]

    agg = pd.concat(cols, axis=1)
    agg.columns = names
    agg.dropna(inplace=True)
    return agg

from sklearn.model_selection import TimeSeriesSplit
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder

dataset = df.values
holiday_encoder = LabelEncoder()
weather_encoder = ColumnTransformer(
    [('1', OneHotEncoder(), [5])],
    remainder='passthrough'
)
dataset[:, 0] = holiday_encoder.fit_transform(dataset[:, 0])
dataset = weather_encoder.fit_transform(dataset)
dataset = pd.DataFrame(dataset)
cols = dataset.columns.tolist()
cols = cols[-1:]+cols[:-1]
dataset = dataset[cols]
dataset = dataset.values
dataset = dataset.astype('float')

n_inputs = 24
n_features = 17
del_idx = n_inputs * n_features + 1
del_cols = [i for i in range(del_idx, del_idx+n_features-1)]
new_df = series_to_img(dataset, n_inputs)
new_df.drop(new_df.columns[del_cols], axis=1, inplace=True)
new_df.head()

,var1(t-24),var2(t-24),var3(t-24),var4(t-24),var5(t-24),var6(t-24),var7(t-24),var8(t-24),var9(t-24),var10(t-24),...,var9(t-1),var10(t-1),var11(t-1),var12(t-1),var13(t-1),var14(t-1),var15(t-1),var16(t-1),var17(t-1),var1(t)
24,5545.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7.0,282.48,0.0,0.0,1.0,5097.0
25,4516.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7.0,291.97,0.0,0.0,1.0,4887.0
26,4767.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7.0,293.23,0.0,0.0,1.0,5337.0
27,5026.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7.0,294.31,0.0,0.0,1.0,5692.0
28,4918.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7.0,295.17,0.0,0.0,1.0,6137.0


In [5]:
n_splits = 10
train_test_split = TimeSeriesSplit(n_splits=n_splits+1, gap=n_inputs).split(new_df)
next(train_test_split)

configs = model_config()
history = []

best_rmse, best_mse, best_mae = [], [], []
learning_time = []
i = 1

print('config : epochs, batch_size, learning_rate')

# neted cross validation
for train_cv_indices, test_cv_indices in train_test_split:
    print(f'fold : {i}/{n_splits}')
    i+=1

    # split x, y data
    train_cv_X, train_cv_y = new_df.iloc[train_cv_indices, :-1].values, new_df.iloc[train_cv_indices,-1].values
    test_cv_X, test_cv_y = new_df.iloc[test_cv_indices, :-1].values, new_df.iloc[test_cv_indices, -1].values

    # length for validation set
    test_length = int(len(train_cv_X)*0.2)

    # scaling data
    scaler_x = MinMaxScaler()
    train_cv_X = scaler_x.fit_transform(train_cv_X)
    test_cv_X = scaler_x.transform(test_cv_X)

    train_X, val_X = train_cv_X[:-test_length, :], train_cv_X[-test_length:, :]
    train_y, val_y = train_cv_y[:-test_length], train_cv_y[-test_length:]

    # reshape
    # inner loop
    train_X = train_X.reshape(-1,  n_inputs, n_features)
    val_X = val_X.reshape(-1, n_inputs, n_features)
    train_y = train_y.reshape(-1, 1)
    val_y = val_y.reshape(-1, 1)

    # outer loop
    train_cv_X = train_cv_X.reshape(-1,  n_inputs, n_features)
    test_cv_X = test_cv_X.reshape(-1, n_inputs, n_features)
    train_cv_y = train_cv_y.reshape(-1, 1)
    test_cv_y = test_cv_y.reshape(-1, 1)

    # model fit, inner
    errors = []
    for idx, cfg in enumerate(configs):
        print(f' == train {cfg} model == ', end=' ')
        model = model_fit(train_X, train_y, cfg)
        model.reset_state()
        predicted = model.predict(val_X)
        if GPU:
            predicted = np.asnumpy(predicted)
        error = np.sqrt(MSE_metric(predicted, val_y))   # rmse
        print(f' error(RMSE):{error}')
        if errors:
            if error < min(errors):
                param = idx
        else:
            param = idx
        errors.append(error)

    history.append(errors)
    
    # check start time
    start_time = time.time()
    # model fitting
    selected_model = model_fit(train_cv_X,train_cv_y, configs[param])
    # check duration
    duration = time.time()-start_time
    selected_model.reset_state()
    predicted = selected_model.predict(test_cv_X)
    if GPU:
        predicted = np.asnumpy(predicted)

    rmse = np.sqrt(MSE_metric(predicted, test_cv_y))
    mse = MSE_metric(predicted, test_cv_y)
    mae = MAE_metric(predicted, test_cv_y)
    best_rmse.append(rmse)
    best_mse.append(mse)
    best_mae.append(mae)
    learning_time.append(duration)

    # model eval
    print(f'train-size:{train_X.shape[0]}, val-size:{val_X.shape[0]}, test-size:{test_cv_X.shape[0]}')
    print(f'best_model => error(rmse) : {rmse.item():.2f}, param:{configs[param]}, times : {duration:.3f}')
    print()

config : epochs, batch_size, learning_rate
fold : 1/10
 == train [30, 64, 0.01] model ==   error(RMSE):2079.520121766977
 == train [30, 64, 0.001] model ==   error(RMSE):2072.8096874774633
train-size:6405, val-size:1601, test-size:4015
best_model => error(rmse) : 2739.93, param:[30, 64, 0.001], times : 155.290

fold : 2/10
 == train [30, 64, 0.01] model ==   error(RMSE):2648.199635568466
 == train [30, 64, 0.001] model ==   error(RMSE):2637.2946376909726
train-size:9617, val-size:2404, test-size:4015
best_model => error(rmse) : 2783.51, param:[30, 64, 0.001], times : 227.669

fold : 3/10
 == train [30, 64, 0.01] model ==   error(RMSE):2031.9461760805445
 == train [30, 64, 0.001] model ==   error(RMSE):2827.6689772059945
train-size:12829, val-size:3207, test-size:4015
best_model => error(rmse) : 2783.78, param:[30, 64, 0.01], times : 305.253

fold : 4/10
 == train [30, 64, 0.01] model ==   error(RMSE):2775.44167376606
 == train [30, 64, 0.001] model ==   error(RMSE):2754.265546008869
tr

In [6]:
def model_evaluation(mse, rmse, mae):
    mse = np.array(mse)
    rmse = np.array(rmse)
    mae = np.array(mae)
    print(f'MSE: mean={np.mean(mse)}, std={np.std(mse)}')
    print(f'RMSE: mean={np.mean(rmse)}, std={np.std(rmse)}')
    print(f'MAE: mean={np.mean(mae)}, std={np.std(mae)}')

model_evaluation(best_mse, best_rmse, best_mae)

# check time
print()
print('[training time]')
print(f'mean : {np.mean(np.array(learning_time))}, last:{learning_time[-1]}')

MSE: mean=7582117.636138921, std=197331.8001445659
RMSE: mean=2753.3304817446633, std=35.901175950389934
MAE: mean=2236.276486525713, std=28.966095233644214

[training time]
mean : 494.46303215026853, last:829.0261769294739
